In [ ]:
==================================
:fa:`cogs` Gulp.js: asset pipeline
==================================

Introduction
============

In this example, I'll show how to make an asset pipeline for a single
Django app that will do the following:

* concatenate multiple CSS files, write out the un-minified CSS to the ``built/``
  directory, then minify the CSS and output the minified CSS as a ``.min.css``
  file with the same name.

  * Note: we'll also re-base any URLs to linked images etc. in the source CSS files,
    re-basing them in the output ``built/`` directory where the combined CSS file
    will be written, so as not to break any related file links.

* concatenate multiple JS files into multiple different *"bundles"* for
  different platforms (mobile and desktop), output the concatenated files to
  the ``built/`` directory, then run *uglifyJS* on the files to minify/obfusticate
  them, writing these files out as ``.min.js`` in the ``built/`` directory too.

* setup *JSHint* to run over the Javascript code in the app that you've written
  (excluding any library code) and outputting nicely formatted *lint* warnings
  that should be addressed in your Javascript code before commit/deployment.

* setup a ``clean`` task (similar to a ``makefile``) to remove any built files
  from the system (will be run as a dependency of the build tasks above).

*...these tasks represent things you'd commonly want to do within the context of
a single app.*

.. note::
   Details concerning *SASS* compilation into CSS are not included here. See
   the guide on :doc:`compile-sass` for details on that score.


Step 1: Set up your Gulp environment
------------------------------------

If you have not used Gulp before, :doc:`set up your environment by following this guide <setup-gulp>`.


Step 2: Install gulp and the plugins
------------------------------------

* In the terminal, ``cd`` into the root of your Django app

* Now, refer to the `gulp plugin directory <http://gulpjs.com/plugins/>`_ to
  find the plugins you'd like to use for your asset pipeline.  For this
  example, we'll use the following:

  * ``gulp`` - the master package
  * ``del`` - deletes files (used for cleaning)
  * ``gulp-concat`` - concatenates files together
  * ``gulp-css-rebase-urls`` - rebases URLs in CSS files
  * ``gulp-rename`` - renames files
  * ``gulp-minify-css`` - minifies CSS
  * ``gulp-uglify`` - runs uglifyJS to minify/obfusticate Javascript
  * ``merge-stream`` - allows multiple streams to converge
  * ``gulp-jshint`` - lint for Javascript
  * ``jshint-stylish`` - styled output from JSHint

* Install the plugins with the following command:

.. code-block:: none

   sudo npm install gulp del gulp-concat gulp-css-rebase-urls gulp-rename gulp-minify-css gulp-uglify merge-stream gulp-jshint jshint-stylish --save

.. note::

   * In the set up guide, you may remember that we installed gulp. We're installing ``gulp`` 
     *again* in order to add it's requirement into the ``package.json`` file.

   * The ``--save`` flag on the end of the command means that after
     installation, the installed items are added into the requirements list
     stored in ``package.json``.  By default, the versions listed in the
     ``package.json`` file will be prefixed by ``^``, meaning
     *"install this version or higher"*.  If you specify a ``--save-exact``
     flag, you can freeze the exact versions instead.

   * If you need to install any plugins at a later date into the pipeline,
     you can run the same command, but just list the new plugins, e.g.
     ``sudo npm install gulp-some-plugin --save``.

   * **If you're not creating an asset-pipeline**, merely using an existing
     one, you just need to run the command ``npm install`` in the same directory
     as the ``package.json`` file (this would be the equivalent of doing
     ``pip install -r requirements.txt`` on a pre-existing project).

   * Installed node-modules live in the ``node_modules/`` directory at the same
     level as ``package.json`` when you run ``npm install``.  **Add** ``node_modules/*`` **to your** ``.gitignore`` **file!**

*...That's it, gulp is now installed. Now move on to making a gulpfile
file that will tell gulp where your files are and what to do with them.*


Step 3: Setup a gulpfile
------------------------

If you've followed the :doc:`guide to setting up your Gulp environment <setup-gulp>` , you should have a blank ``gulpfile.js`` in the root of your Django app. If not, go and do it now. 

The ``gulpfile`` files you'll see around the web will typically follow a common
3-section structure, we'll set them up one at a time.  The 3 sections are:

* **Plugins section:** load the plugins you've installed in step 2, ready for use.
* **Paths section:** a section dedicated to listing/bundling CSS/JS resources relative
  to the ``gulpfile`` file.
* **Tasks section:** where we'll defined named tasks (runnable from the command-line)
  that string together processing pipelines consisting of *gulp* plugins loaded in
  the plugins section that will process the files listed in the path section as required.


Step 3.1: Plugins section
^^^^^^^^^^^^^^^^^^^^^^^^^

This section is trivially easy.  Make a ``var`` statement at the top of the
file, and use ``require()`` to pul in all the plugins installed in Step 2,
storing them in local Javascript variables.

Here's a plugin section that pulls in all the plugins installed in Step 2:

.. code-block:: javascript

   var gulp = require('gulp'),
       merge = require('merge-stream'),
       del = require('del'),
       minify_css = require('gulp-minify-css'),
       rename = require('gulp-rename'),
       rebase = require('gulp-css-rebase-urls'),
       concat = require('gulp-concat'),
       uglify = require('gulp-uglify'),
       jshint = require('gulp-jshint'),
       stylish = require('jshint-stylish');

.. note::
   The string-names passed to ``require()`` must match the installed package names
   you installed in step 2.

   You can call the Javascript variables whatever you like, but you should
   keep them largely the same as the package-name for clarity.

*...simples!*


Step 3.2: Paths section
^^^^^^^^^^^^^^^^^^^^^^^

Suppose your app's static file structure looks like this:

.. code-block:: none

   myapp/
      node_modules/
         ...
      static/
         myapp/
            css/
               styles.css
            img/
               bg.jpg
               icons.png
            js/
               common.js
               common2.js
               desktop.js
               mobile.js
            lib/
               clippy.css
               clippy.js
      __init__.py
      apps.py
      gulpfile
      models.py
      package.json
      urls.py
      views.py

What we want to do is:

* bundle together ``styles.css`` and ``clippy.css`` into the one file, ``myapp.css``

* bundle the Javascript into 2 different packages:

  * ``myapp_desktop.js`` consisting of a bundling of ``clippy.js``, ``common.js``,
    ``common2.js`` and ``desktop.js``.
  * ``myapp_mobile.js`` consisting of a bundling of ``clippy.js``, ``common.js``,
    ``common2.js`` and ``mobile.js``.

* run *JSHint* to lint all of the Javascript code written by us, excluding the
  library code (that wasn't written by us, so we have no maintenance obligation).

The CSS is simple, it's just a list of 2 files.

The Javascript is slightly more complicated.  You'd be best advised to partition
the files into usage categories as outlined by the goals above:

* desktop and mobile Javascript needs separating
* common Javascript is shared between desktop and mobile
* library code should be separated from code we've written (for *JSHint* purposes)

Given the requirements above, lets define our sub-bundles as follows:

.. code-block:: javascript

   var css = [
       './static/myapp/css/styles.css',
       './static/myapp/lib/clippy.css'
   ],
   js_common = [
       './static/myapp/js/common.js',
       './static/myapp/js/common2.js'
   ],
   js_lib = [
       './static/myapp/lib/clippy.js'
   ],
   js_desktop = [
        './static/myapp/js/desktop.js'
   ],
   js_mobile = [
        './static/myapp/js/mobile.js'
   ];

...defining these sub-bundles, and making them javascript arrays means that this
structure is extremely flexible if we need to add new files into the build
at a later date...merely pick the categorisation of the file *(is it library or
something written by us?, is it common to all platforms, or desktop/mobile
specific?, etc.)* and place the new Javascript file into the appropriate list.

Now we'll define the paths and destination folders that will be used by the
tasks we'll define next.  This is done as follows:

.. code-block:: javascript

   var paths = {

       // CSS (all platforms)
       css: {
           src: css,
           dest: './static/myapp/built/'
       },

       // Desktop Javascript
       js_desktop: {
           src: [].concat(js_lib, js_common, js_desktop),
           dest: './static/myapp/built/'
       },

       // Mobile Javascript
       js_mobile: {
           src: [].concat(js_lib, js_common, js_mobile),
           dest: './static/myapp/built/'
       },

       // JSHint Javascript (only code written by us, excludes libraries)
       jshint: {
           src: [].concat(js_common, js_desktop, js_mobile),
       }
   };

.. note::

   * each object under ``paths`` consists of a list of ``src`` files and
     a ``dest`` folder relative to the ``gulpfile``.

   * *...except the* ``jshint`` *object, since JSHint will not output a file,
     it will output code-analysis to the terminal instead.*

   * using the ``concat()`` method of the Javascript ``Array`` class, we can
     construct our final Javascript bundles by plugging together our
     sub-bundles defined earlier.

   * the order that files are listed in these ``src`` attributes is the order
     that the files will appear in the final concatenated built files.

*...that's it, onward to define some tasks!*


Step 3.3 Tasks section
^^^^^^^^^^^^^^^^^^^^^^

clean task
""""""""""

Let's start with the ``clean`` tasks...they're the easiest to explain:

.. code-block:: javascript

   gulp.task('clean_css', function(cb) {
       del(['./static/myapp/built/*.css'], cb);
   });

   gulp.task('clean_js', function(cb) {
       del(['./static/myapp/built/*.js'], cb);
   });

   gulp.task('clean', ['clean_css', 'clean_js']);

Explanation:

* We have a clean task to remove the JS and CSS files in the ``built/`` directory
  individually, and a combined ``clean`` task to do both the JS and CSS

* The ``clean_css`` and ``clean_js`` tasks uses the ``del`` plugin we installed
  earlier.  This is simply a plugin that gets passed an array of file-paths
  relative to the ``gulpfile`` that may include wildcards. Any files matching
  the passed paths will be deleted.

* This also serves as your first introduction to the ``gulp.task()`` function:

  * The first parameter to ``gulp.task()`` is the name of the task.  This should be
    unique within a single ``gulpfile`` and is the name by which you can run
    the task from the command-line.  For e.g., after doing the above, we could
    run ``gulp clean_js`` from the command-line and all the Javascript files
    in the ``built/`` directory would be deleted from the system.

  * The 2nd parameter is optional.  You can see it's omitted in the ``clean_css``
    and ``clean_js`` tasks.  It is present in the ``clean`` task however.

    This list (if specified) is a list of named tasks defined within the same
    ``gulpfile`` that are *"dependencies"* of the task being defined.

    Being a dependency merely means that any tasks appearing in this list will
    be run and completed **before** the task itself is run.

    In the case of the ``clean`` task, you can see that we've combined the
    ``clean_css`` and ``clean_js`` tasks under the single name ``clean``, then
    do nothing else.  Running ``gulp clean`` at the command-line will run the
    ``clean_css`` and ``clean_js`` tasks as dependencies, then exit.

  * The last parameter passed to ``gulp.task()`` should be a function that
    runs when the task is executed.

    *This is also optional...if omitted, as is the case in the* ``clean`` *task,
    the task will do nothing once it's dependencies have run.*

    The ``cb`` parameter passed is optional, it can be ignored most of the
    time.  In this case, it serves as a callback function inside *gulp* that
    will be called when the task is complete. We simply pass it through to
    the ``del`` plugin...that will call the callback once it's deleted all the
    files it needs to.

    The normal flow of a ``gulp.task()`` function is to return the *"stream"*
    from the task-function which acts as a promise-like object that is used to
    signal completion without the need for an explicit callback function to
    be called.

    The ``clean`` tasks are non-typical in this regard.


css task
""""""""

Lets move onto the CSS build task, it looks like this:

.. code-block:: javascript

   gulp.task('css', ['clean_css'], function() {
       return gulp.src(paths.css.src)
           .pipe(rebase({root: paths.css.dest}))
           .pipe(concat('myapp.css'))
           .pipe(gulp.dest(paths.css.dest))
           .pipe(rename({suffix: '.min'}))
           .pipe(minify_css())
           .pipe(gulp.dest(paths.css.dest))
   });

OK, let's start from the top:

* The ``css`` task builds the CSS, before it runs, you can see it has one
  dependency, the ``clean_css`` task will be run before the task-function is
  called to remove any previously built CSS that might be hanging around.

* Once the CSS is cleaned, the task-function will be executed.

* The first line serves as your first introduction to streams:

  * As mentioned above, *gulp* is a stream-based task-runner.

    Unlike *grunt*, which also defines processing pipelines, *gulp* streams
    files byte-by-byte from the first link in the chain to the next and so-on,
    all processing stages happen in parallel, concurrently.

    This is what makes *gulp* massively faster than *grunt*. *grunt* chooses to
    process each individual pipeline stage to completion, before handing over the
    intermediate result to the next stage.  Nothing happens in parallel.

  * The first line, ``gulp.src()`` is fed the list of file-paths defined in
    Step 3.2, in the case of the ``css`` task, the CSS file-paths.

    It opens each file in-turn and begins streaming it's bytes into the
    next stage in the processing pipeline until it's finished passing all files.

    For e.g. in the case of the 2 CSS files, ``styles.css`` and ``clippy.css``,
    the stream will look like this:

    * *<start of styles.css>*
    * *<bytes of styles.css>*
    * *<end of styles.css>*
    * *<start of clippy.css>*
    * *<bytes of clippy.css>*
    * *<end of clippy.css>*

    Usage of the ``pipe()`` method is the thing connects the streaming output
    of one stage to the input sink of the next stage...i.e. that is the *"glue"*
    that holds the pipeline together.

* Before we move onto concatenation, we **must** rebase any URLs in the source
  CSS files, rewriting them relative to the estination ``built/`` directory.

  We need to do this now, since, if we left it until after the ``concat()`` stage,
  ``concat()`` would've removed all knowledge of the original file paths (which
  is vital information to any re-basing process).

* The next processing stage, ``concat()`` is the stage that concatenates the
  CSS.  It will receive all the stream from ``rebase()``, but modifies it to produce
  a single file-stream with the name passed into ``concat()``, e.g.:

    * *<start of myapp.css>*
    * *<bytes of styles.css>*
    * *<separator character(s) - as required>*
    * *<bytes of clippy.css>*
    * *<end of myapp.css>*

* The next stage of the processing is ``dest()``. ``dest()`` is used to write
  the stream out to a destination directory.  We pass it the configured output
  directory for the ``css`` object in ``paths``, i.e. the ``built/`` directory.

  We'll now have a concatenated (but not yet minified) CSS file written out
  to ``myapp/static/built/myapp.css``.

  The ``dest()`` stage also streams the written bytes out the other side too,
  so we can continue streaming if we want to (which we do), ``dest()`` **is not
  terminal.**

* Next, the stream moves onto ``rename()``.  Now we've written the un-minified
  CSS file in the previous stage, we want to write a minified version of it
  to a separate ``.min.css`` file.

  The ``rename()`` module does this for us.  We ask it to add ``.min`` as a
  suffix onto the filename (before the ``.css`` extension).

  This gives us a stream that looks like:

    * *<start of myapp.min.css>*
    * *<bytes of styles.css>*
    * *<separator character(s) - as required>*
    * *<bytes of clippy.css>*
    * *<end of myapp.min.css>*

* Next up, it's the ``minify_css()`` module.  As you might expect, this
  receives CSS, processes it in order to minify it (remove comments, whitespace,
  optimize rules to the smallest short-forms, etc.).  It spits out the bytes of
  minified-CSS out the other end under the same filename as the stream that went
  in.

* Finally, we write out the minified CSS file into the ``built/`` directory using
  ``dest()`` again, like we did before.  This time ``dest()`` is terminal, since
  we've done all the processing we need to.

*...Wow that took a while to explain!  Read it over again to make sure you fully
understand how the streaming works. Remember, all stages of this pipeline happen
in parallel!*


js task
"""""""

Let's move onto the javascript:

.. code-block:: javascript

   gulp.task('js', ['clean_js'], function() {
       return merge(

           gulp.src(paths.js_desktop.src)
               .pipe(concat('myapp_desktop.js'))
               .pipe(gulp.dest(paths.js_desktop.dest))
               .pipe(rename({suffix: '.min'}))
               .pipe(uglify())
               .pipe(gulp.dest(paths.js_desktop.dest)),

           gulp.src(paths.js_mobile.src)
               .pipe(concat('myapp_mobile.js'))
               .pipe(gulp.dest(paths.js_mobile.dest))
               .pipe(rename({suffix: '.min'}))
               .pipe(uglify())
               .pipe(gulp.dest(paths.js_mobile.dest))
       );
   });

OK, this one should be easier to explain:

* Like the ``css`` task, the ``js`` task has a single dependency, ``clean_js``
  that clears out any old Javascript files from the ``built/`` directory before
  starting to build the Javascript in the task-function.
  
* First, take a look at the 2 sub-pipelines in the task-function.  These are
  virtually identical to the ``css`` task explained above, so I won't go into any
  detail about them.  They work **exactly** the same, apart from the ``minify_css()``
  stage being replaced by ``uglify()`` that does the analogous minification on
  Javascript rather than CSS.
  
  There's two pipelines since we have two targets to build Javascript for,
  desktop and mobile.
  
* The interesting part here is the fact that we have two pipelines, not one
  as we did in the ``css`` task.
  
  However, we need to return one thing from the task-function to serve as a
  promise that notifies *gulp* that the task has completed.
  
  And how does *"two become one"* (as a spice-girl might say)?  We use the
  ``merge()`` plugin.  ``merge()`` takes one or more streams as parameters
  and returns a combined stream.  This single combined stream is what we
  return to *gulp*... simple!

.. note::
   We could've done the two Javascript pipelines as separate tasks, however
   that would've meant more tasks to define and name, more clean methods,
   etc.

   I consider it better to have a small number of tasks.


lint task
"""""""""

OK, now let's address the *JSHint* task:

.. code-block:: javascript

   gulp.task('lint', function() {
       return gulp.src(paths.jshint.src)
           .pipe(jshint())
           .pipe(jshint.reporter(stylish));
   });

This should all look familiar to you by now:

* The ``lint`` task has no dependencies, it runs straight away.

* The task-function queues up all the Javascript source-files written by
  us using the ``src()`` method and the list of files we set up way back
  in Step 3.2.

* It feeds the Javascript into ``jshint()`` that will then analyse it.

* Unlike other file-modification pipelines, the output of ``jshint()`` is
  not the contents of a file, but rather the console-output of any
  warning/error messages arising from code-analysis of the Javascript fed
  into it.

* We could just output this straight to the console as-is, however it's
  *rather utilitarian*.  Instead, we choose to pump the output of ``jshint()``
  into the ``jshint-stylish`` plugin.  This will colorize and prettify
  the output, making it a little easier on the eye!

*...Q: are we done yet!?*

*...A: not quite, there's a couple more little tricks that will be of interest
to you...*


default and watch tasks
"""""""""""""""""""""""
.. code-block:: javascript

   gulp.task('default', ['css', 'js']);

   gulp.task('watch', ['default'], function() {
       gulp.watch(paths.css.src, ['css']);
       gulp.watch([paths.js_desktop.src, paths.js_mobile.src], ['js']);
   });

* As mentioned above, you can run any named task defined in a ``gulpfile``
  by running ``gulp <name-of-task>`` at the command-line.

* If you define a task called ``default`` in the ``gulpfile``, this task
  will be run if you just run ``gulp`` at the command-line.

* In our case, we've set up the default task to run both ``css`` and ``js``
  as dependencies, then exit.  i.e. it will build *everything*.  This is
  typically what you'd want the ``default`` task to do.
  
* We've also set up a task called ``watch``.  If, while you're developing,
  you run ``gulp watch`` at the command-line, any changes to the CSS files
  listed in ``paths`` will result in the ``css`` task automatically being
  run to rebuild the combined CSS files in ``built/``.  The same applies to
  the Javascript too.
  
  By making a ``watch`` task, you free yourself from having to run ``gulp``
  on the command-line whenever you change any of the component source-files.
  

Completed gulpfile
^^^^^^^^^^^^^^^^^^

For reference:

.. code-block:: javascript

   // *** PLUGINS ***

   var gulp = require('gulp'),
       merge = require('merge-stream'),
       del = require('del'),
       minify_css = require('gulp-minify-css'),
       rename = require('gulp-rename'),
       concat = require('gulp-concat'),
       rebase = require('gulp-css-rebase-urls'),
       uglify = require('gulp-uglify'),
       jshint = require('gulp-jshint'),
       stylish = require('jshint-stylish');


   // *** PATHS ***

   var css = [
       './static/myapp/css/styles.css',
       './static/myapp/lib/clippy.css'
   ],
   js_common = [
       './static/myapp/js/common.js',
       './static/myapp/js/common2.js'
   ],
   js_lib = [
       './static/myapp/lib/clippy.js'
   ],
   js_desktop = [
        './static/myapp/js/desktop.js'
   ],
   js_mobile = [
        './static/myapp/js/mobile.js'
   ];

   var paths = {

       // CSS (all platforms)
       css: {
           src: css,
           dest: './static/myapp/built/'
       },

       // Desktop Javascript
       js_desktop: {
           src: [].concat(js_lib, js_common, js_desktop),
           dest: './static/myapp/built/'
       },

       // Mobile Javascript
       js_mobile: {
           src: [].concat(js_lib, js_common, js_mobile),
           dest: './static/myapp/built/'
       },

       // JSHint Javascript (only code written by us, excludes libraries)
       jshint: {
           src: [].concat(js_common, js_desktop, js_mobile),
       }
   };


   // *** TASKS ***

   gulp.task('clean_css', function(cb) {
       del(['./static/myapp/built/*.css'], cb);
   });

   gulp.task('clean_js', function(cb) {
       del(['./static/myapp/built/*.js'], cb);
   });

   gulp.task('clean', ['clean_css', 'clean_js']);

   gulp.task('css', ['clean_css'], function() {
       return gulp.src(paths.css.src)
           .pipe(rebase({root: paths.css.dest}))
           .pipe(concat('myapp.css'))
           .pipe(gulp.dest(paths.css.dest))
           .pipe(rename({suffix: '.min'}))
           .pipe(minify_css())
           .pipe(gulp.dest(paths.css.dest))
   });

   gulp.task('js', ['clean_js'], function() {
       return merge(

           gulp.src(paths.js_desktop.src)
               .pipe(concat('myapp_desktop.js'))
               .pipe(gulp.dest(paths.js_desktop.dest))
               .pipe(rename({suffix: '.min'}))
               .pipe(uglify())
               .pipe(gulp.dest(paths.js_desktop.dest)),

           gulp.src(paths.js_mobile.src)
               .pipe(concat('myapp_mobile.js'))
               .pipe(gulp.dest(paths.js_mobile.dest))
               .pipe(rename({suffix: '.min'}))
               .pipe(uglify())
               .pipe(gulp.dest(paths.js_mobile.dest))
       );
   });

   gulp.task('lint', function() {
       return gulp.src(paths.jshint.src)
           .pipe(jshint())
           .pipe(jshint.reporter(stylish));
   });

   gulp.task('default', ['css', 'js']);

   gulp.task('watch', ['default'], function() {
       gulp.watch(paths.css.src, ['css']);
       gulp.watch([paths.js_desktop.src, paths.js_mobile.src], ['js']);
   });
